# Music recommender

### Importing libraries

In [1]:
import config

In [2]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pprint

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

# The "sp" variable has two useful funtions:
# The first usefull function is:
# .search(q='',limit=n)
# .search(q="track:"+song_name+" artist:"+artist_name,limit=5) to restrict to a song name and artist.
# Where the "q" keyword is the query you want to perform on spotify: song_name, artist,...
# while The "limit" keyword will limit the number of returned results.
#
# The second usefull function is:
# .audio_features([URL|URI|ID])
# which returns some 'features of the song', that after cleanup, we can use in order to characterize a song.


In [3]:
import numpy as np
import pandas as pd
import pickle
from sklearn import datasets # sklearn comes with some toy datasets to practice
from sklearn.preprocessing import StandardScaler as scaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sys

In [5]:
soup = BeautifulSoup(url, 'html.parser') 
response = requests.get(url)
response.status_code

NameError: name 'url' is not defined

In [ ]:
import numpy as np
import pandas as pd
import pickle
from sklearn import datasets # sklearn comes with some toy datasets to practice
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score

In [ ]:
recom_music = pd.read_csv('music_final.csv')

In [ ]:
recom_music = recom_music.drop(['id'], axis = 1)
recom_music = recom_music.drop(['Unnamed: 0'], axis = 1)
recom_music = recom_music.drop(['index'], axis =1)

In [ ]:
recom_music

In [ ]:
X = recom_music 

In [ ]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)
display(X.head())
print()
display(X_scaled_df.head())

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(X_scaled_df)

In [ ]:
kmeans.labels_

In [ ]:
labels = kmeans.labels_

In [ ]:
np.unique(labels)

In [ ]:
# assign a cluster to each example
labels = kmeans.predict(X_scaled_df)
# retrieve unique clusters
clusters = np.unique(labels)
# create scatter plot for samples from each cluster
for cluster in clusters:
    # get row indexes for samples with this cluster
    row_ix = np.where(labels == cluster)
    # create scatter of these samples
    pyplot.scatter(X.to_numpy()[row_ix, 1], X.to_numpy()[row_ix, 3])
    # show the plot
pyplot.show()

In [ ]:
clusters = kmeans.predict(X_scaled_df)
#clusters
pd.Series(clusters).value_counts().sort_index()

In [ ]:
#X_df = pd.DataFrame(X)
X["cluster"] = clusters
X

In [ ]:
music_final = pd.read_csv('music_final.csv')
music_id = music_final['id']
music_id

In [ ]:
X = pd.read_csv('music_clustered_with_id.csv')
X = X.drop(['Unnamed: 0'], axis = 1)
X

###  Music player

In [ ]:
track_id=results["tracks"]["items"][0]["id"]
track_id

In [ ]:
from IPython.display import IFrame

#track_id = "1rfORa9iYmocEsnnZGMVC4"
#track_id= 'spotify:track:3hgl7EQwTutSm6PESsB7gZ'
IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [ ]:
def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [ ]:
play_song("track_id")

### DF for the top 100 hot songs

In [ ]:
Bilboard_100 = pd.read_csv('Bilboard_100.csv')
titles = Bilboard_100['Song_Title']
artists = Bilboard_100['Artist_Name']

In [ ]:
low_titles = all_lower(titles)
low_artists = all_lower(artists)

### DF for songs from dif genres

In [ ]:
X = pd.read_csv('music_clustered_with_id.csv')
X = X.drop(['Unnamed: 0'], axis =1)
X

# The song recommender

In [ ]:
# The recommender
#user inputs a song or artist
#is the song hot? if yes - recommend another from the top 100 list 
#if no: 

#should be able to handle typos, upper and lower cases or understand string thats incomplete (fuzzy wuzzy) 

In [ ]:
def all_lower(list):
    return [x.lower() for x in list]

In [ ]:
def get_audio_feature(x):
    find_a_song = sp.search(x, limit=1, market="GB")
    song_features_dict_df = pd.DataFrame(sp.audio_features(find_a_song["tracks"]["items"][0]["id"]))
    song_features_dict_df = song_features_dict_df.drop(['key', 'mode', 'type', 'uri', 'track_href', 'analysis_url', 'time_signature', 'id'], axis=1)
    return song_features_dict_df

In [ ]:
def hot_song(x):
    song_id = sp.search(x, limit=1, market="GB")
    return song_id

In [ ]:
hot_song("Flower")["tracks"]["items"][0]["id"]

In [ ]:
import random
from thefuzz import fuzz
from thefuzz import process



In [ ]:
song_choice = input("Please type the name of a song: ").lower()


for song in titles + low_titles + artists + low_artists:
    if fuzz.partial_ratio(song_choice, song.lower()) >= 80:
        another_song = random.choice(titles)        
        song_player = hot_song(another_song)["tracks"]["items"][0]["id"]
        break
        
    else:
        # get the audio feature by running a function, calling user_input
        song_features_dict_df = get_audio_feature(song_choice)
        # transform them
        song_features_scaled = scaler.transform(song_features_dict_df)
        song_features_scaled_df = pd.DataFrame(song_features_scaled, columns = song_features_dict_df.columns)
        # predict
        cluster = kmeans.predict(song_features_scaled_df)
        cluster = cluster[0]
        # recommend it
         # get random song
        random_song_with_same_cluster = X[(X['cluster'] == cluster)].sample(n=1)
         # get the id of this song
        song_player= str(random_song_with_same_cluster['id'].values[0])
        # play it
        
play_song(song_player)
